CS524: Introduction to Optimization Lecture 33
======================================

## Michael Ferris<br> Computer Sciences Department <br> University of Wisconsin-Madison

## November 18, 2024
--------------

In [1]:
import gamspy as gp
import gamspy.math as gpm
from gamspy import Sum, Card

import sys
import numpy as np
import pandas as pd

## <font color="brown"> What is a cone? </font>

- A set of points $C$ $\in$ $\mathbb{R}^n$ is called a **cone** if it satisfies:
    - $\alpha x \in C $ whenever $x \in C$ and $\alpha > 0$
    
- A cone is convex if in addition
    - $x + y \in C$ whenever $x \in C$ and $y \in C$
    
- Similar to a subspace, but $\alpha > 0$ instead of $\alpha \in \mathbb{R}$ (this is a critical difference!)

- Simple examples: $|x| \leq y$ and $y \geq 0$

<img src="fig1.png">

## <font color="brown"> What is a cone? </font>

- A **<font color="blue"> slice </font>** of a cone is its intersection with a subspace. </br>

- We are interested in **<font color="blue">convex cones</font>** (all slices are convex). </br>

- Can be polyhedral, ellipsoidal or something else... </br>

- Polyhedral cones will have polyhedron slices, ellipsoidal cones will have ellipsoid slices and so on.

<img src="fig2.png">

## <font color="brown"> What is a cone? </font> </font>

### Polyhedral cone recipe:

**1.**  Begin with your favorite polyhedron $Ax ≤ b$ where $x \in \mathbb{R}^n$. This is a polyhedron in $\mathbb{R}^n$ dimensional space.

**2.**  $\{Ax ≤ bt, t ≥ 0\}$ is a polyhedral cone in $(x,t) \in \mathbb{R}^{n+1}$. (parametrized by t involves stacking different sized polyhedron along t-dimension).

**3.**  The slice $t = 1$ is the original polyhedron.

<img src="fig3.png">

## <font color="brown"> Second-order cone (SOC) </font>

A second-order cone is the set of points $x \in \mathbb{R}^n$ satisfying

$$\lVert Ax+b \rVert \leq c^Tx + d$$

Special cases:
- If $A = 0$,  we have a linear constraint (hyperplane)
- If $c= 0$,  can square both sides (ellipsoid) <br><br>

<font color="blue"><center>In general, you <b>cannot</b> just square both sides!</center> </font>

## <font color="brown"> Second-order cone </font>

A second-order cone is the set of points $x \in \mathbb{R}^n$ satisfying

$$\lVert Ax-b \rVert \leq c^Tx + d$$

**Counter example**:

If $A=\begin{bmatrix}1 & 0\end{bmatrix}$ and $c=\begin{bmatrix}0 \\ 1\end{bmatrix}$ and $b = d = 0$, we have:

$$\lvert x \rvert \leq y$$

Square both sides, we get a nonconvex quadratic constraint!

$$x^2 − y^2 \leq 0$$

So, squaring both sides of a convex constraint leads to a nonconvex constraint. !!

<font color="blue"><center>In general, you <b>should NOT</b> just square both sides!</center> </font>

## <font color="brown"> Special case:  rotated second-order cone (Rotated SOC) </font>

A rotated second-order cone is the set $x \in \mathbb{R}^n , y,z\in \mathbb{R}$:

$$x^Tx \leq yz, \;\: y \geq 0,\;\: z \geq 0$$

With $n = 1$, this looks like:

<img src="fig5.png"  width="400" height="400" >

Rotated SOC is linear transformation of SOC, hence the convexity is retained.

## <font color="brown"> Special case:  rotated second-order cone </font>

A rotated second-order cone is the set $x \in \mathbb{R}^n , y,z\in \mathbb{R}$:

$$x^Tx \leq yz, \:\: y \geq 0,\:\:z \geq 0$$

Can put into standard form:
$$4x^Tx \leq 4yz $$
$$4x^Tx + y^2 + z^2 \leq 4yz + y^2 + z^2 $$
$$4x^Tx + (y-z)^2 \leq (y + z)^2 $$
$$\sqrt{4x^Tx + (y-z)^2} \leq y + z $$
$$\left\|\begin{bmatrix}
2x \\
y-z
\end{bmatrix}\right\| \leq y + z $$

## <font color="brown"> Implementation details </font>

A second-order cone program (SOCP) has the form:

$$\min_{x} c^Tx $$ 
$$\text{subject to: } \left\| A_ix + b_i \right\| \leq c_i^Tx + d_i, \text{ for } i = 1,.....,m $$

- Every LP is an SOCP (just make each $A_i = 0$)
- Every convex QP and QCQP is an SOCP
    - convert quadratic cost to epigraph form (add a variable)
    
        $$ \min z \text{ s.t. } z \geq x^TP_0x + r_0^Tx + s_0$$
    
    - convert quadratic constraints to SOCP (complete square as above)
- In GAMSPy, you can specify SOCP constraint directly in QCP. Solvers include Mosek, Gurobi, Cplex.

# Modeling second order cones in GAMSPy

## Example

$$ \max w \text{ s.t. } w + y + z = 1, (w,y,z) \in \mathcal{C}^2_q, (w,y,z) \in \mathcal{C}^1_r(2) $$

Second order cone:
$$\newcommand{\real}{\mathbb{R}} \newcommand{\norm}[1]{\Vert #1 \Vert}
\mathcal{C}^n_q := \left\{(x,z) \in \real^n \times \real : z \geq \norm{x}_2 = \sqrt{\sum_{j=1}^n x_j^2} \right\}$$

Note that 
$$\newcommand{\real}{\mathbb{R}} \newcommand{\norm}[1]{\Vert #1 \Vert}
\mathcal{C}^n_q = \left\{(x,z) \in \real^n \times \real : z \geq 0, \; z^2 \geq \sum_{j=1}^n x_j^2 \right\}$$
so in GAMSPy we write this as (sonc2):

z = m.addVariable('z','positive')\
gpm.sqr(z) >= gpm.sqr(w) + gpm.sqr(y)

Rotated second order cone:
$$\newcommand{\real}{\mathbb{R}} \newcommand{\norm}[1]{\Vert #1 \Vert}
\mathcal{C}^n_r(\alpha) := \left\{(x,y,
z) \in \real^n  \times \real^2_+ : \alpha yz \geq \norm{x}_2^2 = x^T x = \sum_{j=1}^n x_j^2 \right\}$$
so in GAMSPy we write this (with set j being a singleton) as (sonc3):

y = m.addVariable('y','positive')\
z = m.addVariable('z','positive')\
2 * y * z >= gpm.sqr(w)

In [2]:
m = gp.Container()

w = m.addVariable('w')
y = m.addVariable('y','positive')
z = m.addVariable('z','positive')

sonc1 = m.addEquation('sonc1')
sonc1[:] =  w + y + z == 1

# need z >= 0 to convey SOC
sonc2 = m.addEquation('sonc2')
sonc2[:] = gpm.sqr(z) >= gpm.sqr(w) + gpm.sqr(y)

# need y,z >= 0 to convey rotated SOC
sonc3 = m.addEquation('sonc3')
sonc3[:] = 2*y*z >= gpm.sqr(w)

simple = m.addModel('simple',
    equations=m.getEquations(),
    problem=gp.Problem.QCP,
    sense=gp.Sense.MAX,
    objective=w
)

simple.solve(solver='gurobi')

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,0.391576932004034,3,3,QCP,GUROBI,0.032


Mosek requires rotated cones to have \$2 y z\$ in its definition.
Other solvers allow the cone to have 
$$\alpha yz \text{ for any } \alpha > 0 . $$

Mosek needs cones to be a Cartesian product.
Since in above x, y and z appear in both cones, we need to make copies of these variables and use the copies in one of the cones for Mosek.

In [3]:
wc = m.addVariable('wc')
yc = m.addVariable('yc','positive')
zc = m.addVariable('zc','positive')

sonc3a = m.addEquation('sonc3a')
sonc3a[:] = 2*yc*zc >= gpm.sqr(wc)

ew = m.addEquation('ew')
ew[:] = wc == w

ey = m.addEquation('ey')
ey[:] = yc == y

ez = m.addEquation('ez')
ez[:] = zc == z

simpmosek = m.addModel('simpmosek',
    equations=[sonc1, sonc2, sonc3a, ew, ey, ez],
    problem=gp.Problem.QCP,
    sense=gp.Sense.MAX,
    objective=w
)

simpmosek.solve(solver='mosek')

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,0.391577331931458,6,6,QCP,MOSEK,0.006


Can use <= instead of >= in cone definitions.

Some solvers allow expressions in the cone definitions, i.e. we can write the constraint 
$$ (f(x),z) \in \mathcal{C}^n_q$$
where \$f(x)\$ is a linear function of \$x\$.

This is solver dependent however (e.g. following does not work with mosek, gurobi).   Instead you define new variables $v = w + u$ in an additional linear constraint (and rescale rotated cone so constant is 2 for mosek).

In [4]:
u = m.addVariable('u')

# need z >= 0 to convey SOC
sonc2b = m.addEquation('sonc2b')
sonc2b[:] = gpm.sqr(w) + gpm.sqr(y) <= gpm.sqr(z)

# need y,z >= 0 to convey rotated SOC (here alpha = 1)
# f(w,u) = w + u
sonc3b = m.addEquation('sonc3b')
sonc3b[:] = gpm.sqr(w+u) <= y*z

cons = m.addEquation('cons')
cons[:] = w + u >= -5

simpleb = m.addModel('simpleb',
    equations=[sonc1, sonc2b, sonc3b, cons],
    problem=gp.Problem.QCP,
    sense=gp.Sense.MAX,
    objective=w
)

simpleb.solve(solver='cplex')

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,0.499999997148428,4,4,QCP,CPLEX,0.005
